# Managing AI Engines (Driverless AI, H2O)

This notebook is intended to help you get started managing your AI Engines in the H2O AI Cloud using Python.

* **Product Documentation:** https://h2oai.github.io/ai-engine-manager/
* **Python Documentation:** https://h2oai.github.io/ai-engine-manager/docs/py/clients/

In [1]:
import os
import uuid

import h2o
import h2o_authn
import h2o_discovery
import h2o_engine_manager

## Securely connect

We first connect to the H2O AI Cloud by creating a token provider object. We can then use this object to log into AI Engine Manager and other APIs.

In [2]:
discovery = h2o_discovery.discover()

token_provider = h2o_authn.TokenProvider(
    refresh_token=os.getenv("PLATFORM_TOKEN"),
    issuer_url=discovery.environment.issuer_url,
    client_id=discovery.clients["platform"].oauth2_client_id,
)

In [3]:
engine_manager = h2o_engine_manager.login(token_provider=token_provider)

## Managing Driverless AI

In [4]:
dai_manager = engine_manager.dai_engine_client

### View available versions

In [5]:
dai_manager.list_all_versions()

[{'aliases': ['latest'], 'annotations': {}, 'version': '1.10.5'},
 {'aliases': [], 'annotations': {}, 'version': '1.10.4.3'},
 {'aliases': [], 'annotations': {}, 'version': '1.10.4.2'},
 {'aliases': [], 'annotations': {}, 'version': '1.10.4.1'},
 {'aliases': [], 'annotations': {}, 'version': '1.10.4'}]

### Create an engine
See the `3 AutoML` tutorial for what to do once you've connected to the AutoML platform. 

In [7]:
dai_engine = dai_manager.create_engine(
    display_name="My test engine",
    engine_id="test-" + str(uuid.uuid4())[:8], # random engine id
)

dai_engine.wait()

In [8]:
dai_engine

{'annotations': {},
 'api_url': 'https://enginemanager.cloud-dev.h2o.ai/workspaces/default/daiEngines/test-eae58cd2',
 'client_info': <h2o_engine_manager.clients.dai_engine.client_info.ClientInfo object at 0x7fdcf9d3d060>,
 'config': {},
 'cpu': 2,
 'create_time': datetime.datetime(2023, 5, 9, 20, 59, 35, tzinfo=tzlocal()),
 'creator': 'users/56449fb7-31a5-4f4e-b05d-e66aba884bfe',
 'creator_display_name': 'ondrej.bilek@h2o.ai',
 'delete_time': None,
 'display_name': 'My test engine',
 'engine_id': 'test-eae58cd2',
 'gpu': 0,
 'login_url': 'https://enginemanager.cloud-dev.h2o.ai/workspaces/default/daiEngines/test-eae58cd2/oidc/login',
 'max_idle_duration': '1h',
 'max_running_duration': '4h',
 'memory_bytes': '8Gi',
 'name': 'workspaces/default/daiEngines/test-eae58cd2',
 'reconciling': False,
 'resume_time': datetime.datetime(2023, 5, 9, 20, 59, 35, tzinfo=tzlocal()),
 'state': <DAIEngineState.STATE_RUNNING: 'STATE_RUNNING'>,
 'storage_bytes': '8Gi',
 'uid': '4c2cecec-7acf-472d-94d5-90

In [9]:
dai_engine.login_url

'https://enginemanager.cloud-dev.h2o.ai/workspaces/default/daiEngines/test-eae58cd2/oidc/login'

In [10]:
dai = dai_engine.connect()
dai.server.version

'1.10.5'

### List my engines

In [ ]:
for engine in dai_manager.list_all_engines():
    print(engine.name, "\t", engine.state.name, "\t", engine.display_name)

### Download engine logs

In [11]:
dai_engine.download_logs()[:1000] # show only first 1000 characters

"2023-05-09 20:59:49 BEGIN: run.sh\n---------------------------------\nWelcome to H2O.ai's Driverless AI\n---------------------------------\n\n- Put data in the volume mounted at /data\n- Logs are written to the volume mounted at /dai-data/log\n- Connect to Driverless AI on port 12345 inside the container\n2023-05-09 20:59:49 Begin running run-dai.sh\n2023-05-09 20:59:49 BEGIN: run-dai.sh\n2023-05-09 20:59:49 BEGIN: DAI ENV setup\n  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\r  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0\ncurl: (28) Connection timed out after 1001 milliseconds\nNormal setup of conda\n2023-05-09 20:59:50 END: DAI ENV setup\n2023-05-09 20:59:50 Running:\n2023-05-09 20:59:50 BEGIN: start-dai.sh\n2023-05-09 20:59:50 Setting ulimits\n2023-05-09 20:5

### Pause an engine

You can pause an engine that is currently running. Pausing an engine shuts it down to save resources, it is similar to powering off a server. You will not loose any data and you can resume the engine at any time.

In [12]:
dai_engine.pause()
dai_engine.wait()

### Resume the engine

You can resume a paused engine by running:

In [13]:
dai_engine.resume()
dai_engine.wait()

### Delete the engine

When you no longer need an engine, you can terminate it. Once deleted, there is no way to restart the engine or access any data.

In [14]:
dai_engine.delete()
dai_engine.wait()

In [ ]:
for engine in dai_manager.list_all_engines():
    print(engine.name, "\t", engine.state.name, "\t", engine.display_name)

## Managing H2O-3

In [15]:
h2o_manager = engine_manager.h2o_engine_client

### View available versions

In [16]:
h2o_manager.list_all_versions()

[{'aliases': ['latest'], 'annotations': {}, 'version': '3.38.0.3'},
 {'aliases': [], 'annotations': {}, 'version': '3.36.1.5'}]

### Create a H2O engine
See the H2O-3 Tutorial for what to do once you've connected to the distributed ML platform. 

In [17]:
h2o_engine = h2o_manager.create_engine(
    display_name="My test engine",
    engine_id="test-" + str(uuid.uuid4())[:8], # random engine id
    version="latest"
)

h2o_engine.wait()

In [18]:
h2o_engine.login_url

'https://enginemanager.cloud-dev.h2o.ai/workspaces/default/h2oEngines/test-7f448e05/flow/index.html'

In [19]:
h2o.connect(config=h2o_engine.get_connection_config())

Connecting to H2O server at https://enginemanager.cloud-dev.h2o.ai:443/workspaces/default/h2oEngines/test-7f448e05 ... successful.


H2O_cluster_uptime:,14 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,5 months and 16 days !!!
H2O_cluster_name:,test-7f448e05
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.469 Gb
H2O_cluster_total_cores:,1
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


<H2OConnection to https://enginemanager.cloud-dev.h2o.ai:443/workspaces/default/h2oEngines/test-7f448e05, no session>

### List my H2O engines

In [20]:
for engine in h2o_manager.list_all_engines():
    print(engine.name, "\t", engine.state.name, "\t", engine.display_name)

workspaces/default/h2oEngines/test-7f448e05 	 STATE_RUNNING 	 My test engine


### Download H2O logs

In [21]:
h2o_engine.download_logs()[:1000] # show only first 1000 characters

"WARNING: An illegal reflective access operation has occurred\nWARNING: Illegal reflective access by ai.h2o.xgboost4j.java.NativeLibLoader (file:/opt/h2oai/h2o-3/h2o.jar) to field java.lang.ClassLoader.usr_paths\nWARNING: Please consider reporting this to the maintainers of ai.h2o.xgboost4j.java.NativeLibLoader\nWARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations\nWARNING: All illegal access operations will be denied in a future release\n21:43:28.735 [main] INFO  hex.tree.xgboost.util.NativeLibrary - Loaded library from lib/linux_64/libxgboost4j_gpu.so (/ephemeral/libxgboost4j_gpu6760282705797699447.so)\n21:43:28.758 [main] INFO  water.k8s.H2OCluster - Starting Kubernetes-related REST API services\n21:43:28.834 [main] INFO  water.k8s.H2OCluster - Kubernetes REST API services successfully started.\n21:43:28.834 [main] INFO  water.k8s.H2OCluster - Initializing H2O Kubernetes cluster\n21:43:28.835 [main] INFO  water.k8s.H2OCluster - 'H2O_N

### Stop an engine
H2O engines cannot be resumed after they are stopped. Only turn off your cluster once you are permanately done with it.

In [ ]:
h2o_engine.delete()
h2o_engine.wait()

In [ ]:
for engine in h2o_manager.list_all_engines():
    print(engine.name, "\t", engine.state.name, "\t", engine.display_name)